# Step13, 가변 길이 인수(역전파 편)

## 13.1 가변 길이 인수에 대응한 Add 클래스의 역전파

덧셈의 역전파 = 출력 쪽에서 전해지는 미분값 x 1 = 입력 변수(x0,x1)의 미분  
즉 상류에서 흘러오는 미분값을 '그대로 흘려보내는 것'이 덧셈의 역전파

In [102]:
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [103]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [104]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y


def add(x0, x1):
    return Add()(x0, x1)


x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


In [105]:
# Add 클래스 구현 

class Add(Function):
    def forward(self,x0,x1):
        y = x0 + x1 
        return y
    
    def backward(self,gy):
        return gy, gy

def add(x0,x1):
    return Add()(x0,x1)


위와 같이 backward 메서드는 입력이 1개, 출력이 2개  
여러개의 값을 반환하기 위해 Variable 클래스에서 backward 메서드를 수정해 본다.

In [106]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            # x, y = f.input, f.output
            gys = [output.grad for output in f.outputs] # 1
            gxs = f.backward(*gys)                      # 2
            if not isinstance(gxs,tuple):               # 3
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs):            # 4
                x.grad = gx 
            # x.grad = f.backward(y.grad)

                if x.creator is not None:
                    funcs.append(x.creator)

1,2,3,4 를 추가 

1. 출력 변수인 outputs에 담겨 있는 미분값들을 리스트에 담는다.
2. 함수 f의 역전파를 호출한다. 이때 f.backward(*gys)처럼 인수에 별표를 붙여 호출한여 리스트 언팩한다.
3. gxs가 튜플이 아니면 튜플로 리턴한다.
4. 역전파로 전파되는 미분값(gx)을 Variable의 인스턴스 변수 grad(x.grad)에 저장  
    - gxs와 f.inputs의 각 원소는 서로 대응 관계에 있다. 따라서 i번째 원소에 대해 f.inputs\[i\]의 미분값 : gxs\[i\] 대응
    - zip 함수와 for문을 이용해서 모든 Variable 인스턴스 각각에 알맞은 미분값을 설정


In [107]:
Number = [1,2,3,4]
Name = ['hong','gil','dong','nim']
Number_Name = list(zip(Number,Name))
print(Number_Name)

[(1, 'hong'), (2, 'gil'), (3, 'dong'), (4, 'nim')]


In [108]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

## 13.3 Square 클래스 구현

In [109]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        # x = self.input.data
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

def square(x):
    f = Square()
    return f(x)

add 함수와 square 함수를 이용하여 $$z = x^2 + y^2$$ 를 계산하는 코드

In [110]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


최종 코드

In [ ]:
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):                            # 입력받는 데이터가 ndarray 구조가 아니면 오류 발생
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data        # 데이터 선언
        self.grad = None        # 미분값 선언
        self.creator = None     # 이 데이터가 어디출신인지, 어느 공장에서 만들어졌는지 표기

    def set_creator(self, func):    # 생성자 = 공장 = 함수
        self.creator = func 

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)     # 미분값이 없으면 모두 1로 구성된 행렬

        funcs = [self.creator]                      # 함수들을 담는 리스트 
        while funcs:
            f = funcs.pop()                         # 함수들을 하나씩 뽑는다.
            gys = [output.grad for output in f.outputs]     # 출력변수인 outputs에 담겨있는 미분값(.grad)들을 리스트에 담는다
            gxs = f.backward(*gys)                          # f의 역전파를 호출한다. *를 붙혀 리스트를 풀면서 넣어준다.(리스트 언팩)
            if not isinstance(gxs, tuple):                  # gxs가 튜플이 아니면 튜플로 변환한다.
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):                # gxs와 f.inputs의 각 원소는 서로 대응 관계
                x.grad = gx                                 # 역전파로 전파되는 미분값을 Variable의 인스턴스 변수 grad에 저장

                if x.creator is not None:
                    funcs.append(x.creator)



def as_array(x):
    if np.isscalar(x):
        return np.array(x)                                  # 입력이 스칼라인 경우 ndarray 인스턴스로 변화해줌
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]                       # 가변길이 인수를 다루기위해, 변수를 리스트에 담아 취급
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]       # 가변길이 입력이므로 가변길이 출력을 리스트로 담는다

        for output in outputs:
            output.set_creator(self)                        # 각각의 output들이 어디 출신 변수인지 정해짐, 자신이 창조자라고 원산지 표시를 함
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]  # 리스트의 원소가 하나라면 첫번째 원소를 반환한다, 해당 변수를 직접 돌려준다

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    f = Square()
    return f(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)


x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)